In [1]:
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest, chi2_contingency, ttest_ind



# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("Data/raw_data_final_20231212.csv", index_col=0)

In [3]:
df.head(2)

,Age,Attrition,Businesstravel,Department,Education,Educationfield,Employeenumber,Environmentsatisfaction,Gender,Jobinvolvement,Joblevel,Jobrole,Jobsatisfaction,Maritalstatus,Monthlyrate,Numcompaniesworked,Overtime,Percentsalaryhike,Relationshipsatisfaction,Stockoptionlevel,Trainingtimeslastyear,Yearsatcompany,Yearssincelastpromotion,Yearswithcurrmanager,Datebirth,Remotework,Dailyrate,Distancefromhome,Monthlyincome,Performancerating,Totalworkingyears,Worklifebalance,Hourlyrate
0,51,No,Unknown,Research & Development,3,Unknown,162,Not Satisfied,Male,High,Executive,Research Director,Satisfied,Unknown,6462,7,No,13,Satisfied,0,5,20,15,15,1972,Yes,684.0,6.0,19537.0,3.0,13.6,3.0,51.0
1,52,No,Unknown,Unknown,4,Life Sciences,259,Satisfied,Male,Moderate,Executive,Manager,Satisfied,Unknown,5678,0,Unknown,14,Not Satisfied,1,5,33,11,9,1971,Yes,699.0,1.0,19999.0,3.0,34.0,3.0,65.0


# Clasificación de los registros por grupos según la satisfacción

In [4]:
def category_satisfaction(value):
    if (value=="Satisfied") or (value=="Very Satisfied"):
        return 'A'
    else:
        return 'B'

In [5]:
df['Groups'] = df['Jobsatisfaction'].apply(category_satisfaction)

In [6]:
df["Groups"]

0       A
1       A
2       A
3       A
4       B
       ..
1609    A
1610    B
1611    A
1612    A
1613    A
Name: Groups, Length: 1510, dtype: object

# Cálculo de los porcentajes de "ATTRITION"

La métrica principal que debes usar es la tasa de rotación de empleados (Attrition) en cada grupo. Recordemos que, en esta columna, "No" indica que el empleado no ha dejado la empresa (sin rotación), mientras que "Yes" indica que el empleado ha dejado la empresa (con rotación).
Por lo tanto, los pasos que deberás seguir en esta fase son:
Divide a los empleados en los grupos A y B según los criterios establecidos.
Calcula la tasa de rotación (porcentaje de empleados que dejaron la empresa) en cada grupo.

In [10]:
group_a= df[df["Groups"]=="A"]

In [16]:
group_a.shape

(925, 34)

In [17]:
group_a["Attrition"].value_counts()/group_a.shape[0]*100

No     86.27027
Yes    13.72973
Name: Attrition, dtype: float64

In [18]:
group_b= df[df["Groups"]=="B"]

In [19]:
group_b.shape

(585, 34)

In [20]:
group_b["Attrition"].value_counts()/group_b.shape[0]*100

No     80.512821
Yes    19.487179
Name: Attrition, dtype: float64

In [22]:
rotacion_por_grupo = df[df['Attrition'] == 'Yes'].groupby('Groups').size() / df.groupby('Groups').size() * 100

In [25]:
rotacion_por_grupo

Groups
A    13.729730
B    19.487179
dtype: float64

# Prueba Estadística

Comprobar los tres supuestos para pruebas paramétricas: 
    Evalúa la normalidad de una columna de datos de un DataFrame utilizando la prueba de Shapiro-Wilk.

    Parámetros:
        dataframe (DataFrame): El DataFrame que contiene los datos.
        columna (str): El nombre de la columna en el DataFrame que se va a evaluar para la normalidad.

    Returns:
        None: Imprime un mensaje indicando si los datos siguen o no una distribución normal.
    """

    statistic, p_value = stats.shapiro(dataframe[columna])
    if p_value > 0.05:
        print(f"Para la columna {columna} los datos siguen una distribución normal.")
    else:
        print(f"Para la columna {columna} los datos no siguen una distribución normal.")

In [26]:
# Crear una tabla de contingencia
contingency_table = pd.crosstab(df['Attrition'], df['Groups'])

# Ejecutar el test de chi-cuadrado
chi2, p_valor, _, _ = chi2_contingency(contingency_table)

# Mostrar los resultados del test
print("\nResultados del test de Chi-cuadrado:")
print(f"Estadística de Chi-cuadrado: {chi2}")
print(f"P-valor: {p_valor}")

# Interpretación del resultado
alpha = 0.05  # Nivel de significancia
if p_valor < alpha:
    print("\nHay evidencia suficiente para rechazar la hipótesis nula.")
else:
    print("\nNo hay suficiente evidencia para rechazar la hipótesis nula.")


Resultados del test de Chi-cuadrado:
Estadística de Chi-cuadrado: 8.432334983446875
P-valor: 0.0036860670797145547

Hay evidencia suficiente para rechazar la hipótesis nula.
